# MNI Calculation

This notebook prepares data for MNI calculation and writes the results to the `source_data.xlsx` file.

In [ ]:
import pandas as pd
import sys
sys.path.append('..')
from scripts.mni import calculate_mni

df_occurrences = pd.read_pickle("../data/pkl/df_occurrences.pkl")
df_briana_with_responses = pd.read_pickle("../data/pkl/df_briana_with_responses.pkl")

columns = [
    "TransectUID",
    "Taxon Label",
    "Pre: Age",
    "Pre: Sex",
    "What element is this?",
    "Side",
]

df = pd.concat(
    [
        df_occurrences.reindex(columns=columns),
        df_briana_with_responses.reindex(columns=columns),
    ],
    ignore_index=True,
)


In [ ]:
pivot_df = (
    df.pivot_table(
        index=["TransectUID", "Taxon Label", "Pre: Age", "Pre: Sex", "What element is this?"],
        columns="Side",
        aggfunc="size",
        fill_value=0,
    )
    .rename_axis(columns=None)
    .reset_index()
)
pivot_df.head()

In [ ]:
mni_per_transect = calculate_mni(pivot_df)
mni_per_transect

In [ ]:
from pathlib import Path
from openpyxl import load_workbook

output_path = Path("../data/export/excel/source_data.xlsx")
if output_path.exists():
    transect_sheet = pd.read_excel(output_path, sheet_name="Transects")
else:
    transect_sheet = pd.DataFrame()

transect_sheet = transect_sheet.merge(mni_per_transect, on="TransectUID", how="left")
transect_sheet = transect_sheet.rename(columns={"MNI": "MNI_calc"})

output_path.parent.mkdir(parents=True, exist_ok=True)
if output_path.exists():
    book = load_workbook(output_path)
    with pd.ExcelWriter(output_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        writer.book = book
        transect_sheet.to_excel(writer, sheet_name="Transects", index=False)
else:
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        transect_sheet.to_excel(writer, sheet_name="Transects", index=False)